# Solution for Puzzle x

## Imports

In [2]:
import pandas as pd
import os

## Input Data

In [3]:
def read_and_parse_irregular_data(file_path: str) -> pd.DataFrame:
    """
    Reads an irregularly formatted text file line by line, parses the data 
    blocks using string splitting, and returns a consolidated Pandas DataFrame.

    Args:
        file_path: The path to the input file (e.g., 'input/test.csv').

    Returns:
        A pandas DataFrame containing the parsed data.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"Error: File not found at '{file_path}'. Please check the path.")
        
    raw_lines = []
    try:
        # Read the entire file content into a list of strings
        with open(file_path, 'r', encoding='utf-8') as file:
            # Strip whitespace and filter out empty lines
            raw_lines = [line.strip() for line in file if line.strip()]
    except Exception as e:
        raise RuntimeError(f"An error occurred while reading the file: {e}")

    parsed_data = []

    # Parse each line string
    for line in raw_lines:

        # Split the line by spaces to separate the data blocks
        parts = line.split(": ")
        
        # Check if the line contains enough data blocks
        if len(parts) < 2:
            print(f"Skipping line due to insufficient data: {line}")
            continue

        node = parts[0]
        vertices = parts[1].split(" ")
        
        # Prepare the data dictionary for the row
        row_dict = {'node': node, 'vertices': vertices}
        
        parsed_data.append(row_dict)

    # 3. Create the final DataFrame from the list of dictionaries
    df = pd.DataFrame(parsed_data)
    
    return df

file_to_process = 'input/day_11.csv'
input = read_and_parse_irregular_data(file_to_process)

print("\n--- Resulting DataFrame ---")
print(input.head(5))


--- Resulting DataFrame ---
  node              vertices
0  zlo                 [tnx]
1  cwi       [lct, vxu, was]
2  agf            [dac, fob]
3  uqn  [mik, ygx, yrg, zpj]
4  saf                 [ktw]


## Part One

In [ ]:
# Building graph datastructure
graph = {}
for _, row in input.iterrows():
    graph[row["node"]] = row["vertices"]

# Traversing graph starting from you node
paths = set()
stack = [("you", [])]
while stack:
    curr_node, visited = stack.pop(0)
    visited.append(curr_node)
    for vertice in graph[curr_node]:
        if vertice == "out":
            paths.add(tuple(visited))
        elif vertice not in visited:
            stack.append((vertice, visited.copy()))

print(f"There are {len(paths)} possible paths.")

There are 497 possible paths.


## Part Two

In [15]:
from functools import lru_cache

def dfs_memoized(graph):
    """
    Finds the number of paths from 'svr' to 'out' that visit both 'dac' and 'fft'.

    Args:
        graph (dict): A dictionary where keys are device nodes and values are
                      a list of devices they connect to (outgoing connections).
    """

    # Memoized DFS function
    @lru_cache(None) 
    def count_paths(start, end):
        """
        Counts the number of unique directed paths from 'start' to 'end' in the graph.
        """
        if start == end:
            return 1

        path_count = 0
        
        # Recurse on all neighbors
        for neighbor in graph.get(start, []):
            path_count += count_paths(neighbor, end)

        return path_count

    # Define the key nodes required by the puzzle
    START_NODE = 'svr'
    END_NODE = 'out'
    NODE_A = 'fft'
    NODE_B = 'dac'

    # Calculate paths for Case 1: svr -> ... -> NODE_A -> ... -> NODE_B -> ... -> out
    paths_p1 = count_paths(START_NODE, NODE_A)
    paths_p2 = count_paths(NODE_A, NODE_B)
    paths_p3 = count_paths(NODE_B, END_NODE)
    count_case_1 = paths_p1 * paths_p2 * paths_p3
    
    # Calculate paths for Case 2: svr -> ... -> NODE_B -> ... -> NODE_A -> ... -> out
    paths_p4 = count_paths(START_NODE, NODE_B)
    paths_p5 = count_paths(NODE_B, NODE_A)
    paths_p6 = count_paths(NODE_A, END_NODE)
    count_case_2 = paths_p4 * paths_p5 * paths_p6

    total_paths = count_case_1 + count_case_2
    
    count_paths.cache_clear() 
    
    return total_paths

print(f"There are {dfs_memoized(graph)} possible paths.")

There are 358564784931864 possible paths.
